In [1]:
import mysql.connector
import os
from dotenv import load_dotenv
from mysql.connector import Error

import pandas as pd

In [2]:
def get_data_from_db(table_name, columns = '*'):
    
    # dot env
    load_dotenv()
    aws_db_ufo_pw = os.getenv('aws_db_ufo_pw')
    aws_host_ufo = os.getenv('aws_host_ufo')

    remote = mysql.connector.connect(
        host = aws_host_ufo,
        user = 'EDA',
        port = 3306,  # default 는 3306
        password = aws_db_ufo_pw,
        database = 'Frisbee'

        )
        
    

    cursor = remote.cursor(dictionary = True)
    
    
    query = f"""
        SELECT {columns} FROM {table_name}"""
    
    cursor.execute(query)
    
    results = cursor.fetchall()

    
    
    column_name = [des[0] for des in cursor.description]
    
    cursor.close()
    
    df = pd.DataFrame.from_records(results, index = column_name[0])
    
    
    return df
    

In [3]:
df_ufo = get_data_from_db('UFO_us')
df_metheo = get_data_from_db('metheo')
df_date_dim = get_data_from_db('DateDimension')
df_sw = get_data_from_db('space_weather')

### df_ufo_전처리

In [12]:
df_ufo.head()

,date,time,city,state,shape,duration,comments,latitude,longitude
id,,,,,,,,,
1190,1995-01-01,0 days,el granada,ca,fireball,45,Glowing fireball object crashed at Maverick&#...,37.5028,-122.4680
1191,1995-01-01,0 days,cedar key,fl,triangle,180,I could hear and see it&#44 close encounter.,29.1383,-83.0353
1192,1995-01-01,0 days,greenwood,sc,sphere,20,Bright red sphere floating and completely sile...,34.1953,-82.1619
1193,1995-01-01,0 days,tucson,az,triangle,600,120-140 ft&#44 X 35-40 ft&#44 X 30 ft&#44 tria...,32.2217,-110.9260
1194,1995-01-01,0 days,pacific grove,ca,unknown,0,Rept. that local police had pursued a strange ...,36.6178,-121.9160


In [13]:
# groupby 메소드를 통해 판다스 시리즈 형태의 데이터로 전환
df_ufo_gb = df_ufo.groupby('date').size()

In [14]:
# index를 리셋하면서 판다스 시리즈에서 
df_ufo_gb = df_ufo_gb.to_frame()
df_ufo_gb = df_ufo_gb.reset_index()
df_ufo_gb

,date,0
0,1995-01-01,20
1,1995-01-02,2
2,1995-01-03,12
3,1995-01-04,4
4,1995-01-05,10
...,...,...
6798,2014-05-04,8
6799,2014-05-05,14
6800,2014-05-06,12
6801,2014-05-07,12


In [15]:
df_ufo_gb['ufo_count'] = df_ufo_gb[0]
df_ufo_gb = df_ufo_gb[['date', 'ufo_count']]

In [16]:
df_ufo_gb.head()

,date,ufo_count
0,1995-01-01,20
1,1995-01-02,2
2,1995-01-03,12
3,1995-01-04,4
4,1995-01-05,10


In [17]:
df_ufo_gb.dtypes

date         datetime64[ns]
ufo_count             int64
dtype: object

### df_metheo 전처리

In [18]:
df_metheo.head()

,date,time,address,state,latitude,longitude,elevation,magnitude,white,red,blue,green,purple,orange,yellow
id,,,,,,,,,,,,,,,
1,1995-06-08,0 days 02:00:00,"Trafalgar, IN",IN,39,-86,267,-13,0,0,0,1,0,0,0
2,1995-06-20,0 days 17:00:00,"Munhall, PA",PA,40,-79,329,-13,0,1,0,0,0,1,0
3,1995-07-04,0 days 21:30:00,"Myrtle, MS",MS,34,-89,108,-19,0,1,0,0,0,1,1
4,1995-08-01,0 days 16:00:00,"Pottstown, PA",PA,40,-75,46,-16,0,1,0,0,0,0,0
5,1995-12-15,0 days 06:50:00,"Eagan, MN",MN,44,-93,257,-15,1,0,0,0,0,0,0


In [19]:
df_metheo_gb = df_metheo.groupby('date').size()

In [20]:
df_metheo_gb

date
1995-06-08       1
1995-06-20       1
1995-07-04       1
1995-08-01       1
1995-12-15       1
              ... 
2014-12-27      39
2014-12-28      31
2014-12-29    1173
2014-12-30      78
2014-12-31      25
Length: 334, dtype: int64

In [21]:
df_metheo_gb = df_metheo_gb.to_frame()
df_metheo_gb


,0
date,
1995-06-08,1
1995-06-20,1
1995-07-04,1
1995-08-01,1
1995-12-15,1
...,...
2014-12-27,39
2014-12-28,31
2014-12-29,1173


In [22]:
df_metheo_gb = df_metheo_gb.reset_index()
df_metheo_gb

,date,0
0,1995-06-08,1
1,1995-06-20,1
2,1995-07-04,1
3,1995-08-01,1
4,1995-12-15,1
...,...,...
329,2014-12-27,39
330,2014-12-28,31
331,2014-12-29,1173
332,2014-12-30,78


In [23]:
df_metheo_gb['metheo_cnt'] = df_metheo_gb[0]
df_metheo_gb['date'] = df_metheo_gb['date'].astype('str')

In [24]:
df_date_dim = df_date_dim.reset_index()
df_date_dim

,calendar_date
0,1990-01-01
1,1990-01-02
2,1990-01-03
3,1990-01-04
4,1990-01-05
...,...
9126,2014-12-27
9127,2014-12-28
9128,2014-12-29
9129,2014-12-30


In [25]:
df_date_dim.dtypes

calendar_date    object
dtype: object

In [26]:
df_ufo_gb.dtypes

date         datetime64[ns]
ufo_count             int64
dtype: object

In [27]:
df_metheo_gb.dtypes

date          object
0              int64
metheo_cnt     int64
dtype: object

In [28]:
df_date_dim['calendar_date'] = df_date_dim['calendar_date'].astype('str')
df_ufo_gb['date'] = df_ufo_gb['date'].astype('str')
df_metheo_gb['date'] = df_metheo_gb['date'].astype('str')

/var/folders/kh/6khx28zs2bq8wjptklpbkj440000gn/T/ipykernel_6189/2150101569.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ufo_gb['date'] = df_ufo_gb['date'].astype('str')


### ufo데이터와 metheo데이터 날짜 데이터로 병합

In [29]:
merged_ufo_date = pd.merge(
    df_date_dim,
    df_ufo_gb,
    left_on='calendar_date',     # date_dim의 조인 키
    right_on='date',             # metheo_cleaned의 조인 키
    how='outer'                  # Outer Join 지정
)

In [30]:
merged_df = pd.merge(
    merged_ufo_date,
    df_metheo_gb,
    left_on='calendar_date',     # date_dim의 조인 키
    right_on='date',             # metheo_cleaned의 조인 키
    how='outer'                  # Outer Join 지정
)

In [31]:
merged_df

,calendar_date,date_x,ufo_count,date_y,0,metheo_cnt
0,1990-01-01,NaN,NaN,NaN,NaN,NaN
1,1990-01-02,NaN,NaN,NaN,NaN,NaN
2,1990-01-03,NaN,NaN,NaN,NaN,NaN
3,1990-01-04,NaN,NaN,NaN,NaN,NaN
4,1990-01-05,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
9126,2014-12-27,NaN,NaN,2014-12-27,39.0,39.0
9127,2014-12-28,NaN,NaN,2014-12-28,31.0,31.0
9128,2014-12-29,NaN,NaN,2014-12-29,1173.0,1173.0
9129,2014-12-30,NaN,NaN,2014-12-30,78.0,78.0


In [32]:
merged_df.to_csv('ufo_metheo_merged.csv')

### 시각화

In [33]:
from bokeh.plotting import figure, save
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.io import output_file
from sklearn.preprocessing import MinMaxScaler

In [34]:
# 1. 데이터 로드
df = pd.read_csv('/Users/sungminhong/Documents/dev_ws/ufo_eda/data/ufo_metheo_merged.csv')

# 2. 데이터 정리 및 Bokeh 오류 수정
# (a) 컬럼 이름 공백 제거
df.columns = df.columns.str.strip()

# (b) Bokeh 오류 (invalid keys: nan) 해결:
#     필요한 컬럼만 명시적으로 선택하여 새 DataFrame을 만듭니다.
#     이렇게 하면 'nan'이라는 이름의 컬럼이 무시됩니다.
plot_df = df[['calendar_date', 'ufo_count', 'metheo_cnt']].copy()

# (c) NaN 값을 0으로 채웁니다 (Outer Join으로 인해 발생한 NaN)
plot_df['ufo_count'] = plot_df['ufo_count'].fillna(0)
plot_df['metheo_cnt'] = plot_df['metheo_cnt'].fillna(0)

scaler = MinMaxScaler()

plot_df[['ufo_normalized','metheo_normalized']] = scaler.fit_transform(
    plot_df[['ufo_count', 'metheo_cnt']]
)

# (d) 날짜 타입 변환 (Bokeh 축을 위함)
plot_df['calendar_date'] = pd.to_datetime(plot_df['calendar_date'])

# 3. Bokeh ColumnDataSource 생성
#    오류가 해결된 plot_df를 사용합니다.
source = ColumnDataSource(plot_df)

# 4. 출력 파일 설정
output_file("ufo_vs_metheo_time_series.html")

# 5. 시각화 생성 (Figure)
p = figure(
    width=800,
    height=450,
    x_axis_type="datetime",  # x축을 날짜/시간 타입으로 설정
    title="날짜별 UFO 출현 및 운석 보고 횟수",
    x_axis_label="날짜",
    y_axis_label="보고 횟수",
    tools="pan,wheel_zoom,box_zoom,reset,save"
)

# 6. 첫 번째 라인: UFO Count
p.line(
    x='calendar_date',
    y='ufo_normalized',
    source=source,
    line_width=1,
    color="#53DAEB",
    alpha=0.8,
    legend_label="UFO 횟수 (ufo_count)"
)

# 7. 두 번째 라인: Metheo Count
p.line(
    x='calendar_date',
    y='metheo_normalized',
    source=source,
    line_width=1,
    color="#F3BD6D",
    alpha=0.8,
    legend_label="운석 횟수 (metheo_cnt)"
)

# 8. HoverTool (마우스 오버 툴팁) 추가
hover = HoverTool(
    tooltips=[
        ("날짜", "@calendar_date{%Y-%m-%d}"),  # 날짜 형식 지정
        ("UFO 횟수", "@ufo_normalized{0}"),      # 정수로 표시
        ("운석 횟수", "@metheo_normalized{0}"),      # 정수로 표시
    ],
    formatters={"@calendar_date": "datetime"}
)
p.add_tools(hover)

# 9. 범례 설정 (클릭 시 라인 숨기기)
p.legend.location = "top_left"
p.legend.click_policy = "hide"

# 10. HTML 파일로 저장
save(p)

print("Bokeh 시각화가 'ufo_vs_metheo_time_series.html' 파일로 저장되었습니다.")

Bokeh 시각화가 'ufo_vs_metheo_time_series.html' 파일로 저장되었습니다.


### ufo 데이터와 space_weather 데이터의 병합

In [35]:
merged_ufo_date

,calendar_date,date,ufo_count
0,1990-01-01,NaN,NaN
1,1990-01-02,NaN,NaN
2,1990-01-03,NaN,NaN
3,1990-01-04,NaN,NaN
4,1990-01-05,NaN,NaN
...,...,...,...
9126,2014-12-27,NaN,NaN
9127,2014-12-28,NaN,NaN
9128,2014-12-29,NaN,NaN
9129,2014-12-30,NaN,NaN


In [36]:
df_sw

,date,ap_avg,black_dot_cnt,f107_adj
id,,,,
1,1994-01-01,26.0,126,143.4
2,1994-01-02,21.0,113,141.4
3,1994-01-03,15.0,136,128.7
4,1994-01-04,6.0,151,124.6
5,1994-01-05,4.0,153,132.4
...,...,...,...,...
11319,2024-12-27,2.0,257,250.0
11320,2024-12-28,3.0,254,251.8
11321,2024-12-29,4.0,233,246.3


In [38]:
df_sw.dtypes

date             datetime64[ns]
ap_avg                  float64
black_dot_cnt             int64
f107_adj                float64
dtype: object

In [39]:
df_sw['date'] = df_sw['date'].astype('str')

In [40]:
merged_df = pd.merge(
    merged_ufo_date,
    df_sw,
    left_on='calendar_date',     # date_dim의 조인 키
    right_on='date',            
    how='outer'                  
)

In [41]:
merged_df

,calendar_date,date_x,ufo_count,date_y,ap_avg,black_dot_cnt,f107_adj
0,1990-01-01,NaN,NaN,NaN,NaN,NaN,NaN
1,1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN
2,1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN
3,1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN
4,1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
12779,NaN,NaN,NaN,2024-12-27,2.0,257.0,250.0
12780,NaN,NaN,NaN,2024-12-28,3.0,254.0,251.8
12781,NaN,NaN,NaN,2024-12-29,4.0,233.0,246.3
12782,NaN,NaN,NaN,2024-12-30,6.0,218.0,216.1


In [42]:
merged_df['calendar_date'] = pd.to_datetime(merged_df['calendar_date'])

In [45]:
merged_df = merged_df[merged_df['calendar_date'] >='1995-01-01']

In [46]:
merged_df.to_csv('ufo_sw_merged.csv')

In [59]:
color_ufo = "#EE7132"
color_ap = "#6DF393"
color_black_dot = "#6DF3E8"
color_f107 = "#6D6FF3"

In [60]:
merged_df.columns = merged_df.columns.str.strip()

In [61]:
cols_to_plot = ['ufo_count', 'ap_avg', 'black_dot_cnt', 'f107_adj']
date_col = 'calendar_date'
plot_df = merged_df[cols_to_plot + [date_col]].copy()

In [62]:
plot_df[cols_to_plot] = plot_df[cols_to_plot].fillna(0)

In [63]:
plot_df[date_col] = pd.to_datetime(plot_df[date_col])

In [64]:
scaler = MinMaxScaler()
normalized_cols = [f'{c}_norm' for c in cols_to_plot]
# 정규화 적용
plot_df[normalized_cols] = scaler.fit_transform(plot_df[cols_to_plot])

In [65]:
source = ColumnDataSource(plot_df)

# 5. 출력 파일 설정
output_file("ufo_sw_multi_line_plot.html")

# 6. 시각화 생성 (Figure)
p = figure(
    width=1000,
    height=500,
    x_axis_type="datetime",
    title="UFO 및 태양 활동 지표 (0-1 정규화)",
    x_axis_label="날짜",
    y_axis_label="정규화된 값 (0 to 1)",
    tools="pan,wheel_zoom,box_zoom,reset,save"
)
# Y축 범위를 0-1로 고정
p.y_range.start = 0
p.y_range.end = 1

# 7. 다중 라인 추가

# AP Average (Normalized)
p.line(
    x=date_col, y='ap_avg_norm', source=source, line_width=1,
    color=color_ap, alpha=0.8, legend_label="AP 평균"
)
# Black Dot Count (Normalized)
p.line(
    x=date_col, y='black_dot_cnt_norm', source=source, line_width=1,
    color=color_black_dot, alpha=0.8, legend_label="흑점 수"
)
# F10.7 (Normalized)
p.line(
    x=date_col, y='f107_adj_norm', source=source, line_width=1,
    color=color_f107, alpha=0.8, legend_label="F10.7 (조정됨)"
)

# UFO Count (Normalized)
p.line(
    x=date_col, y='ufo_count_norm', source=source, line_width=1,
    color=color_ufo, alpha=0.8, legend_label="UFO 횟수"
)

# 8. HoverTool 추가 (툴팁에는 정규화되기 '전'의 원본 값을 표시)
hover = HoverTool(
    tooltips=[
        ("날짜", f"@{date_col}{{%Y-%m-%d}}"),
        ("UFO 횟수", "@ufo_count{0}"),
        ("AP 평균", "@ap_avg{0.0}"),
        ("흑점 수", "@black_dot_cnt{0}"),
        ("F10.7", "@f107_adj{0.0}"),
    ],
    formatters={f"@{date_col}": "datetime"}
)
p.add_tools(hover)

# 9. 범례 설정
p.legend.location = "top_left"
p.legend.click_policy = "hide"

# 10. HTML 파일로 저장
save(p)

print("Bokeh 다중 라인 시각화가 'ufo_sw_multi_line_plot.html' 파일로 저장되었습니다.")

Bokeh 다중 라인 시각화가 'ufo_sw_multi_line_plot.html' 파일로 저장되었습니다.


In [67]:
plot_df_corr = plot_df[['ufo_count', 'ap_avg', 'black_dot_cnt', 'f107_adj']]

In [68]:
plot_df_corr = plot_df_corr.corr()

In [70]:
from bokeh.plotting import figure, save
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LinearColorMapper,
    ColorBar,
    BasicTicker,
    PrintfTickFormatter
)
from bokeh.transform import transform
from bokeh.palettes import RdBu11  # 상관관계에 적합한 Red(-)/Blue(+) 팔레트
from bokeh.io import output_file

In [72]:
cols_to_corr = ['ufo_count', 'ap_avg', 'black_dot_cnt', 'f107_adj']

plot_df_corr.index.name = 'Var1'
plot_df_corr.columns.name = 'Var2'

# DataFrame으로 변환 (예: Var1, Var2, Correlation)
corr_data = plot_df_corr.stack().reset_index(name='Correlation')
# 텍스트로 표시할 값을 소수점 2자리 문자열로 포맷팅하여 새 컬럼 생성
corr_data['CorrelationText'] = corr_data['Correlation'].apply(lambda x: f"{x:.2f}")

# 컬럼 이름 목록 (x_range, y_range에 사용)
var_names = cols_to_corr

# --- 4. Bokeh 히트맵(Heatmap) 시각화 ---
output_file("correlation_heatmap.html")

# 색상 매퍼(Mapper) 설정
# 상관관계는 -1 (파란색) ~ +1 (빨간색) 범위
# RdBu11 팔레트를 거꾸로 사용 (빨간색이 양수, 파란색이 음수)
palette = list(reversed(RdBu11))
color_mapper = LinearColorMapper(
    palette=palette,
    low=-1,  # 최소값
    high=1   # 최대값
)

# Bokeh Figure 생성
p = figure(
    width=550,
    height=550,
    title="상관관계 분석 히트맵",
    x_range=var_names,
    y_range=list(reversed(var_names)), # Y축은 거꾸로 표시
    x_axis_location="above",
    tools="hover,save" # 호버 툴 추가
)

# 사각형(Rect) 글리프를 사용하여 히트맵 생성
p.rect(
    x="Var2",
    y="Var1",
    width=1,
    height=1,
    source=ColumnDataSource(corr_data),
    fill_color=transform('Correlation', color_mapper), # 상관관계 값에 따라 색상 매핑
    line_color="white" # 셀 경계선
)

# Text 글리프 추가
p.text(
    x="Var2",
    y="Var1",
    text="CorrelationText", # 위에서 생성한 문자열 컬럼 사용
    source=ColumnDataSource(corr_data),
    text_align="center",
    text_baseline="middle",
    text_color="black", # 상관계수 값이 1 또는 -1에 가까울 때 대비
    text_font_size="10pt" # 폰트 크기
)

# 색상 막대(Color Bar) 추가
color_bar = ColorBar(
    color_mapper=color_mapper,
    ticker=BasicTicker(desired_num_ticks=5), # 눈금 수
    formatter=PrintfTickFormatter(format="%.2f"), # 소수점 2자리
    label_standoff=10
)
p.add_layout(color_bar, 'right')

# HoverTool 설정 (마우스 오버 시 값 표시)
# 툴팁에 @ 기호를 사용하여 ColumnDataSource의 컬럼 참조
hover_tool = p.select_one(HoverTool)
hover_tool.tooltips = [
     ('변수 1', '@Var1'),
     ('변수 2', '@Var2'),
     ('상관관계', '@Correlation{%.3f}'), # 소수점 3자리까지 표시
]

# 축 스타일 설정
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.xaxis.major_label_orientation = 0.7 # 라벨 약간 회전

# 5. HTML 파일로 저장
save(p)

print("Bokeh 상관관계 히트맵이 'correlation_heatmap.html' 파일로 저장되었습니다.")

Bokeh 상관관계 히트맵이 'correlation_heatmap.html' 파일로 저장되었습니다.
